In [1]:
### Libraries
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape
from keras.models import Model
from keras import backend as K
import numpy as np

Using TensorFlow backend.


In [2]:
### Loading Data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = np.load(r'bottom_x_train.npy')
x_test = np.load(r'bottom_x_test.npy')

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print('Training data shape: ', x_train.shape)
print('Testing data shape : ', x_test.shape)

Training data shape:  (60000, 28, 28)
Testing data shape :  (10000, 28, 28)


In [3]:
input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
### Training AE
from keras.callbacks import TensorBoard
x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(10000,28,28,1)
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 19s 314us/step - loss: 0.1219 - val_loss: 0.0813
Epoch 2/50
60000/60000 [==============================] - 17s 283us/step - loss: 0.0773 - val_loss: 0.0726
Epoch 3/50
60000/60000 [==============================] - 17s 283us/step - loss: 0.0694 - val_loss: 0.0666
Epoch 4/50
60000/60000 [==============================] - 17s 283us/step - loss: 0.0648 - val_loss: 0.0628
Epoch 5/50
60000/60000 [==============================] - 17s 284us/step - loss: 0.0618 - val_loss: 0.0619
Epoch 6/50
60000/60000 [==============================] - 17s 283us/step - loss: 0.0594 - val_loss: 0.0590
Epoch 7/50
60000/60000 [==============================] - 17s 283us/step - loss: 0.0579 - val_loss: 0.0571
Epoch 8/50
60000/60000 [==============================] - 17s 282us/step - loss: 0.0569 - val_loss: 0.0554
Epoch 9/50
60000/60000 [======================

In [5]:
### Get Encoded features fron AE
encoder = Model(input_img, encoded)
encoded_imgs_test = encoder.predict(x_test)
encoded_imgs_train = encoder.predict(x_train)
print(encoded_imgs_test.shape)
print(encoded_imgs_train.shape)

(10000, 4, 4, 8)
(60000, 4, 4, 8)


In [6]:
encoded_imgs_test[0]

array([[[0.5305677 , 0.        , 0.        , 0.09398005, 0.37799397,
         0.        , 0.63616174, 0.8817668 ],
        [0.50345653, 0.        , 0.40542552, 0.21385351, 0.34523183,
         0.        , 0.5944749 , 0.8567619 ],
        [0.47050312, 0.08358201, 0.06465843, 0.40509298, 0.32315558,
         0.        , 0.896054  , 0.92295945],
        [0.5077153 , 0.        , 0.22658002, 0.15152271, 0.28277388,
         0.        , 0.7598185 , 0.6966809 ]],

       [[0.43489316, 0.        , 0.35688525, 0.19857265, 0.30728057,
         0.        , 0.79810107, 0.8084071 ],
        [0.15602805, 1.5925767 , 1.1906971 , 0.60484105, 0.4833144 ,
         1.2912296 , 1.3187982 , 0.6775946 ],
        [0.        , 1.2280915 , 0.25342157, 1.5116044 , 1.7256677 ,
         0.9689305 , 0.35254234, 1.1924239 ],
        [0.        , 0.22507161, 0.05374591, 0.29361483, 0.21514177,
         0.        , 0.7605541 , 1.0909193 ]],

       [[0.37834102, 0.42194754, 1.187997  , 0.71043456, 0.20972383,
       

In [7]:
encoded_imgs_test = encoded_imgs_test.reshape(10000,128)
encoded_imgs_train = encoded_imgs_train.reshape(60000,128)
print(encoded_imgs_test.shape)
print(encoded_imgs_train.shape)

(10000, 128)
(60000, 128)


In [8]:
np.save('bottom_test.npy',encoded_imgs_test)
np.save('bottom_train.npy',encoded_imgs_train)